In [ ]:
import numpy as np
import torch
ora_W = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_ora_W.npy')
new_W = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_new_W.npy')
test_inps = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_test_inps.npy')
test_outs = np.load('/h3cstore_ns/jcxie/LISA/nips2024/test_npy/9_self_attn.v_proj_test_outs.npy')

ora_W = torch.tensor(ora_W).to(torch.float16).cuda()
new_W = torch.tensor(new_W).to(torch.float16).cuda()
test_inps = torch.tensor(test_inps).to(torch.float16)
test_outs = torch.tensor(test_outs).to(torch.float16)

In [8]:
from tqdm import tqdm
import math
import time



test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0
for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)

    nsamples += 1

    H_A += (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T
    H_B += (math.sqrt(2 / nsamples) * test_outs[i][0].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T

    

XTX = H_A
W_hat = new_W.to(torch.float32).cuda()
S = (W_hat != 0).to(torch.float32).cuda()
W_0 = ora_W.to(torch.float32).cuda()


import torch
from tqdm import tqdm

def vectorized_pcg_solver(XTX, W_0, S, max_iter=100, tol=1e-6, device='cuda'):

    XTX = XTX.to(device)
    W_0 = W_0.to(device)
    S = S.to(device)
    

    R0 = XTX @ W_0
    

    R0 = R0 * S
    
    # 计算预条件子 M = Diag(XTX) 的逆
    M_diag = torch.diag(XTX)
    # 避免除以零，添加小常数
    M_inv_diag = 1.0 / (M_diag + 1e-10)
    M_inv = torch.diag(M_inv_diag)
    
    # 计算 Z0 = M^(-1)R0
    Z0 = M_inv @ R0
    
    # 初始化P0 = Z0
    P0 = Z0.clone()
    
    # 初始化迭代变量
    R = R0
    Z = Z0
    P = P0
    W = torch.zeros_like(W_0)  # 初始解设为零矩阵
    
    # 记录收敛历史
    residuals = []
    
    # 开始迭代
    for t in tqdm(range(max_iter)):
        # 计算 X⊤XPt
        XTX_P = XTX @ P
        
        # 计算 αt = Tr(Rt⊤Zt)/ Tr(Pt⊤X⊤XPt)
        # 注意：在矩阵形式下，Tr(A⊤B) 等价于 A 和 B 的逐元素乘积之和
        numerator = torch.sum(R * Z)
        denominator = torch.sum(P * XTX_P)
        
        # 避免除以零
        if denominator == 0:
            break
            
        alpha = numerator / denominator
        
        # 更新解 Wt+1 = Wt + αtPt
        W = W + alpha * P
        
        # 更新残差 Rt+1 = Rt - αtX⊤XPt
        R_next = R - alpha * XTX_P
        
        # 投影到支撑集S
        R_next = R_next * S
        
        # 计算残差范数，用于收敛检查
        residual_norm = torch.norm(R_next)
        residuals.append(residual_norm.item())
        
        # 检查收敛
        if residual_norm < tol:
            print(f"Converged at iteration {t+1} with residual norm {residual_norm:.6e}")
            break
            
        # 计算 Zt+1 = M^(-1)Rt+1
        Z_next = M_inv @ R_next
        
        # 计算 βt = Tr(Rt+1⊤Zt+1)/ Tr(Rt⊤Zt)
        beta = torch.sum(R_next * Z_next) / (torch.sum(R * Z) + 1e-10)
        
        # 更新搜索方向 Pt+1 = Zt+1 + βtPt
        P = Z_next + beta * P
        
        # 更新迭代变量
        R = R_next
        Z = Z_next
    
    return W, residuals

W_final, residuals = vectorized_pcg_solver(XTX, W_0, S, max_iter=1000, tol=1e-6, device='cuda')



100%|██████████| 1000/1000 [00:25<00:00, 39.63it/s]


In [15]:
import torch
import time
import numpy as np

# （接你提供的代码）
test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0
for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)

    nsamples += 1

    H_A += (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T
    H_B += (math.sqrt(2 / nsamples) * test_outs[i][0].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T

XTX = H_A  # X⊤X矩阵
W_hat = new_W.to(torch.float32).cuda()  # 剪枝后的初始解（带支撑集）
S = (W_hat != 0).to(torch.float32).cuda()  # 支撑集掩码（非零元素位置）
W_0 = ora_W.to(torch.float32).cuda()  # 目标解（论文中的cW）


def pcg_vectorized(XTX, W0, S, max_iter=50, rtol=1e-4, atol=1e-6, verbose=False):

    device = XTX.device
    n, m = W0.shape  # n: 特征维度，m: 批量大小（如输出维度）

    # 初始化解W（初始猜测为W_hat，即带支撑集的初始解）
    W = W_hat.clone()

    # 步骤1: 计算初始残差 R0 = X⊤X (W0 - W)
    R = XTX @ (W0 - W)

    # 步骤2: 投影残差到支撑集S（支撑集外元素置零）
    R = R * S

    # 步骤3: 初始化预条件子M = Diag(XTX)，M_inv = Diag(1/diag(XTX))
    diag_XTX = torch.diag(XTX).unsqueeze(1)  # 扩展为(n×1)，方便广播到(n×m)
    M_inv = 1.0 / (diag_XTX + 1e-10)  # 避免除零
    Z = M_inv * R  # Z0 = M^{-1} R0
    P = Z.clone()  # P0 = Z0

    # 收敛判断阈值
    R0_norm = torch.norm(R)
    stopping_thresh = torch.max(rtol * R0_norm, torch.tensor(atol, device=device))

    # 迭代信息
    optimal = False
    start_time = time.perf_counter()

    if verbose:
        print(f"初始化残差范数: {R0_norm:.6e}，收敛阈值: {stopping_thresh:.6e}")
        print("%03s | %10s | %10s" % ("迭代", "残差范数", "相对误差"))

    for t in range(max_iter):
        # 步骤5: 计算αt = Tr(Rt⊤ Zt) / Tr(Pt⊤ X⊤X Pt)
        # Tr(A⊤B) = 元素乘积之和（对批量矩阵按列求和后累加）
        numerator = torch.sum(R * Z)  # 等价于Tr(R⊤Z)
        XTX_P = XTX @ P
        denominator = torch.sum(P * XTX_P)  # 等价于Tr(P⊤ X⊤X P)
        alpha = numerator / (denominator + 1e-10)  # 避免除零

        # 步骤6: 更新解 Wt+1 = Wt + αt Pt
        W = W + alpha * P

        # 步骤7: 更新残差 Rt+1 = Rt - αt X⊤X Pt
        R = R - alpha * XTX_P

        # 步骤8: 投影残差到支撑集S
        R = R * S

        # 检查收敛
        current_norm = torch.norm(R)
        rel_error = current_norm / (R0_norm + 1e-10)

        if verbose:
            print(f"%03d | %10.6e | %10.6e" % (t+1, current_norm.item(), rel_error.item()))

        if current_norm < stopping_thresh or rel_error < rtol:
            optimal = True
            break

        # 步骤9: 计算Zt+1 = M^{-1} Rt+1
        Z = M_inv * R

        # 步骤13: 计算βt = Tr(Rt+1⊤ Zt+1) / Tr(Rt⊤ Zt)
        beta_numerator = torch.sum(R * Z)
        beta_denominator = numerator + 1e-10  # 复用之前的numerator=Tr(Rt⊤ Zt)
        beta = beta_numerator / beta_denominator

        # 步骤14: 更新搜索方向 Pt+1 = Zt+1 + βt Pt
        P = Z + beta * P

    # 迭代结束
    total_time = time.perf_counter() - start_time
    if verbose:
        if optimal:
            print(f"迭代 {t+1} 次收敛，耗时 {total_time:.3f}s")
        else:
            print(f"达到最大迭代次数 {max_iter}，耗时 {total_time:.3f}s，最终残差: {current_norm:.6e}")

    info = {
        "niter": t+1,
        "optimal": optimal,
        "final_residual": current_norm.item(),
        "time": total_time
    }
    return W, info


# 调用PCG求解
print("\n===== 运行论文中的PCG算法 =====")
W_final, pcg_info = pcg_vectorized(
    XTX=XTX,
    W0=W_0,
    S=S,
    max_iter=500,
    rtol=1e-4,
    verbose=True
)

# 验证结果：检查 X⊤X W ≈ X⊤X W0
XTX_W = XTX @ W_final
XTX_W0 = XTX @ W_0
error = torch.norm(XTX_W - XTX_W0, p='fro') / torch.norm(XTX_W0, p='fro')
print(f"\n求解相对误差: {error.item():.6e}")
print(f"支撑集一致性: {(W_final * (1 - S)).abs().max().item():.6e} (应接近0)")


===== 运行论文中的PCG算法 =====
初始化残差范数: 5.114889e+05，收敛阈值: 5.114889e+01
 迭代 |       残差范数 |       相对误差
001 | 9.219654e+04 | 1.802513e-01
002 | 1.896738e+05 | 3.708268e-01
003 | 1.641573e+05 | 3.209402e-01
004 | 7.987156e+04 | 1.561550e-01
005 | 1.016196e+05 | 1.986741e-01
006 | 1.121187e+05 | 2.192006e-01
007 | 8.343537e+04 | 1.631225e-01
008 | 5.531611e+04 | 1.081472e-01
009 | 6.789459e+04 | 1.327391e-01
010 | 6.695902e+04 | 1.309100e-01
011 | 4.935154e+04 | 9.648602e-02
012 | 4.619877e+04 | 9.032213e-02
013 | 4.494494e+04 | 8.787079e-02
014 | 3.975726e+04 | 7.772849e-02
015 | 3.642823e+04 | 7.121997e-02
016 | 3.335642e+04 | 6.521436e-02
017 | 3.191435e+04 | 6.239499e-02
018 | 3.102917e+04 | 6.066440e-02
019 | 2.776988e+04 | 5.429223e-02
020 | 2.376853e+04 | 4.646929e-02
021 | 2.267966e+04 | 4.434047e-02
022 | 2.502985e+04 | 4.893528e-02
023 | 2.210369e+04 | 4.321440e-02
024 | 1.811390e+04 | 3.541405e-02
025 | 2.247052e+04 | 4.393158e-02
026 | 2.003433e+04 | 3.916866e-02
027 | 1.682755e+04 |

In [16]:
W_final = W_final.to(torch.float16)
print((W_final == 0).sum() / W_final.numel())
for i in [0, 32, 63, 122, 123, 124]:
    # W_final = W_final - M.to(torch.float16) * W_final
    print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print(i, (W_final @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_final @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    print(i, (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # W_update2 = W_update - M.to(torch.float16) * W_update
    # print(i, (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # print("-----------")
    # print((new_W @ test_inps[i]).abs().max(), (new_W @ test_inps[i]).abs().mean())
    # print((W_update2 @ test_inps[i]).abs().max(), (W_update2 @ test_inps[i]).abs().mean())
    # print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print("-------------------------------------------------------------------------")

tensor(0.7000, device='cuda:0')
tensor(3.9316, dtype=torch.float16) tensor(0.2715, dtype=torch.float16)
0 tensor(1.7285, device='cuda:0', dtype=torch.float16) tensor(0.1083, device='cuda:0', dtype=torch.float16)
0 tensor(0.5742, device='cuda:0', dtype=torch.float16) tensor(0.0740, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(5.1016, dtype=torch.float16) tensor(0.2737, dtype=torch.float16)
32 tensor(1.7207, device='cuda:0', dtype=torch.float16) tensor(0.1102, device='cuda:0', dtype=torch.float16)
32 tensor(0.8164, device='cuda:0', dtype=torch.float16) tensor(0.0758, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(3.5605, dtype=torch.float16) tensor(0.2708, dtype=torch.float16)
63 tensor(2.0273, device='cuda:0', dtype=torch.float16) tensor(0.1140, device='cuda:0', dtype=torch.float16)
63 tensor(0.6562, device='cuda:0', dtype=torch.float16) tenso

In [29]:
from tqdm import tqdm
import math
import time

lambda_value = []
lambda_abs = []
alphas = []

out = []

def alpha_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    # if k >= max_iter:
    #     return alpha_end
    ratio = k / max_iter
    return alpha_end - (alpha_end - alpha_start) * np.exp(-gamma * ratio)

def delta_schedule_exp(k, max_iter=100, alpha_start=0.9, alpha_end=0.99, gamma=5.0):
    if k >= max_iter:
        return alpha_end - alpha_start
    else:
        return alpha_schedule_exp(k, max_iter, alpha_start, alpha_end, gamma) - alpha_start

def alpha_schedule(k, max_iter=100, alpha_start=0.9, alpha_end=0.99):
    if k >= max_iter:
        return alpha_end
    else:
        return alpha_start + (alpha_end - alpha_start) * (k / max_iter)

def dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, epsilon=1e-1, max_iter=10, lambda_zero=False, percdamp=.01):
    M = M.to(torch.float32)
    H_A = H_A.to(torch.float32)
    H_B = H_B.to(torch.float32)
    W_old = W_old.to(torch.float32)
    
    # 初始化 W 和 Lambda
    W = W_old.clone()
    if lambda_zero:
        Lambda = torch.zeros_like(W)
    # else:
    #     term1 = beta * (torch.mm(W, H_A) - H_B)
    #     term2 = alpha * (W - W_old)
    #     Lambda = -M * (term1 + term2)
    I = torch.eye(H_A.shape[0], device=H_A.device)
    A = (1-alpha) * H_A + alpha * I

    A_start = torch.linalg.cholesky(A)
    A_start = torch.cholesky_inverse(A_start)
    
    A1 = -(A_start @ (I - H_A) @ A_start)
    A2 = (-A1) @ (I - H_A) @ A_start
    A3 = (-A2) @ (I - H_A) @ A_start


    alpha_start = alpha
    for k in tqdm(range(max_iter)):
        alpha = alpha_schedule_exp(k, alpha_start=alpha_start)
        delta = alpha - alpha_start

        if k % 20 == 0 and k < 100:
            alpha_start = alpha

            A = (1-alpha) * H_A + alpha * I

            A_start = torch.linalg.cholesky(A)
            A_start = torch.cholesky_inverse(A_start)
            
            A1 = -(A_start @ (I - H_A) @ A_start)
            A2 = (-A1) @ (I - H_A) @ A_start
            A3 = (-A2) @ (I - H_A) @ A_start



        W_prev = W.clone()

        B = (1-alpha-delta) * H_B + (alpha + delta) * W_prev
        B = B - Lambda

        # A_inv = (1-alpha-delta) * H_A + (alpha + delta) * I
        # A_inv = torch.linalg.cholesky(A_inv)
        # A_inv = torch.cholesky_inverse(A_inv)

        W = B @ (A_start + delta * A1 + delta**2 * A2 + delta**3 * A3)
        # W = B @ A_inv

        # print("二阶近似误差",torch.norm(((A_start + delta * A1 + delta**2 * A2) - A_inv)))


        Lambda = Lambda + rho * (M * W)

        lambda_value.append(Lambda.mean().item())
        lambda_abs.append(Lambda.abs().mean().item())
        out.append((M * W).mean().item())
        # W = W - M * W
        # 收敛判断

        if k % 50 == 0 and k > 0:
            

            # print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
            # print(torch.norm(W - W_prev))
            if torch.norm(W - W_prev) < epsilon:
                print( (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().max(), (W.to(torch.float16) @ test_inps[0].cuda() - test_outs[0][0].cuda()).abs().mean())
                print(f"Converged at iteration {k}")
                print(torch.norm(W - W_prev))
                break

        # if k % 10 == 0 :
        #     print(torch.norm(W - W_prev))
    W = W - M * W
    return W.to(torch.float16)

test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0
for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)

    nsamples += 1

    H_A += (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T
    H_B += (math.sqrt(2 / nsamples) * test_outs[i][0].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T

    

M = (new_W == 0).to(torch.float32).to(H_B.device)
print(M.device)
# M = torch.zeros_like(M).to(torch.float32).cuda()
print((M == 0).sum() / M.numel())
W_old = new_W.clone().to(H_B.device)
beta = 0.01
alpha = 0.9
gama = 0.0000
rho = 1


start = time.time()

W_update = dual_ascent_method10(H_A, H_B, W_old, M, beta, alpha, gama, rho, lambda_zero=True, max_iter=200)
print('time', time.time() - start)

cuda:0
tensor(0.3000, device='cuda:0')


 75%|███████▌  | 150/200 [00:03<00:01, 41.54it/s]

tensor(0.5771, device='cuda:0', dtype=torch.float16) tensor(0.0676, device='cuda:0', dtype=torch.float16)
Converged at iteration 150
tensor(0.0898, device='cuda:0')
time 3.6276891231536865


In [32]:
B = torch.matmul(ora_W.to(torch.float32), H_A).to(torch.float32).cuda()
print((B-H_B).abs().max(), (B-H_B).abs().mean())

tensor(0.2969, device='cuda:0') tensor(0.0003, device='cuda:0')


In [14]:
print((W_update == 0).sum() / W_update.numel())
for i in [0, 32, 63, 122, 123, 124]:
    # W_update = W_update - M.to(torch.float16) * W_update
    print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print(i, (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    print(i, (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # W_update2 = W_update - M.to(torch.float16) * W_update
    # print(i, (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # print("-----------")
    # print((new_W @ test_inps[i]).abs().max(), (new_W @ test_inps[i]).abs().mean())
    # print((W_update2 @ test_inps[i]).abs().max(), (W_update2 @ test_inps[i]).abs().mean())
    # print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print("-------------------------------------------------------------------------")

tensor(0.7000, device='cuda:0')
tensor(3.9316, dtype=torch.float16) tensor(0.2715, dtype=torch.float16)
0 tensor(0.5762, device='cuda:0', dtype=torch.float16) tensor(0.0679, device='cuda:0', dtype=torch.float16)
0 tensor(0.5742, device='cuda:0', dtype=torch.float16) tensor(0.0740, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(5.1016, dtype=torch.float16) tensor(0.2737, dtype=torch.float16)
32 tensor(0.8398, device='cuda:0', dtype=torch.float16) tensor(0.0696, device='cuda:0', dtype=torch.float16)
32 tensor(0.8164, device='cuda:0', dtype=torch.float16) tensor(0.0758, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(3.5605, dtype=torch.float16) tensor(0.2708, dtype=torch.float16)
63 tensor(0.6514, device='cuda:0', dtype=torch.float16) tensor(0.0791, device='cuda:0', dtype=torch.float16)
63 tensor(0.6562, device='cuda:0', dtype=torch.float16) tenso

In [15]:
print((W_update == 0).sum() / W_update.numel())
for i in [0, 32, 63, 122, 123, 124]:
    # W_update = W_update - M.to(torch.float16) * W_update
    print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print(i, (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    print(i, (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # W_update2 = W_update - M.to(torch.float16) * W_update
    # print(i, (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # print("-----------")
    # print((new_W @ test_inps[i]).abs().max(), (new_W @ test_inps[i]).abs().mean())
    # print((W_update2 @ test_inps[i]).abs().max(), (W_update2 @ test_inps[i]).abs().mean())
    # print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print("-------------------------------------------------------------------------")

tensor(0.7000, device='cuda:0')
tensor(3.9316, dtype=torch.float16) tensor(0.2715, dtype=torch.float16)
0 tensor(0.5762, device='cuda:0', dtype=torch.float16) tensor(0.0679, device='cuda:0', dtype=torch.float16)
0 tensor(0.5742, device='cuda:0', dtype=torch.float16) tensor(0.0740, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(5.1016, dtype=torch.float16) tensor(0.2737, dtype=torch.float16)
32 tensor(0.8398, device='cuda:0', dtype=torch.float16) tensor(0.0696, device='cuda:0', dtype=torch.float16)
32 tensor(0.8164, device='cuda:0', dtype=torch.float16) tensor(0.0758, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(3.5605, dtype=torch.float16) tensor(0.2708, dtype=torch.float16)
63 tensor(0.6514, device='cuda:0', dtype=torch.float16) tensor(0.0791, device='cuda:0', dtype=torch.float16)
63 tensor(0.6562, device='cuda:0', dtype=torch.float16) tenso

In [38]:
import torch.nn.functional as F
import time
import math
def cg_batch(A, B, A_supp, M_bmm=None, X0=None, rtol=1e-3, atol=0., maxiter=None, verbose=False):
    """Solves a batch of PD matrix linear systems using the preconditioned CG algorithm.

    This function solves matrix linear systems of the form

        A X = B,  

    where A is a n x n positive definite matrix and B is a n x m matrix,
    and X is the n x m matrix representing the solution for the ith system.

    Args:
        A_bmm: A callable that performs a batch matrix multiply of A and a n x m matrix.
        B: A n x m matrix representing the right hand sides.
        M_bmm: (optional) A callable that performs a batch matrix multiply of the preconditioning
        matrices M and a n x m matrix. (default=identity matrix)
        X0: (optional) Initial guess for X, defaults to M_bmm(B). (default=None)
        rtol: (optional) Relative tolerance for norm of residual. (default=1e-3)
        atol: (optional) Absolute tolerance for norm of residual. (default=0)
        maxiter: (optional) Maximum number of iterations to perform. (default=5*n)
        verbose: (optional) Whether or not to print status messages. (default=False)
    """
    error_list = np.zeros((maxiter,))
    n, m = B.shape

    if M_bmm is None:
        M_bmm = lambda x: x
    if X0 is None:
        X0 = M_bmm(B)
    if maxiter is None:
        maxiter = 5 * n

    assert B.shape == (n, m)
    assert X0.shape == (n, m)
    assert rtol > 0 or atol > 0
    assert isinstance(maxiter, int)

    X_k = X0

    R_k = B - A @ X_k
    R_k = R_k * A_supp

    Z_k = M_bmm(R_k)

    P_k = torch.zeros_like(Z_k)

    P_k1 = P_k
    R_k1 = R_k
    R_k2 = R_k
    X_k1 = X0
    Z_k1 = Z_k
    Z_k2 = Z_k

    B_norm = torch.norm(B, dim=1)
    stopping_matrix = torch.max(rtol*B_norm, atol*torch.ones_like(B_norm))

    if verbose:
        print("%03s | %010s %06s" % ("it", "dist", "it/s"))

    optimal = False
    start = time.perf_counter()
    for k in range(1, maxiter + 1):
        start_iter = time.perf_counter()
        Z_k = M_bmm(R_k)

        if k == 1:
            P_k = Z_k
            R_k1 = R_k
            X_k1 = X_k
            Z_k1 = Z_k
        else:
            R_k2 = R_k1
            Z_k2 = Z_k1
            P_k1 = P_k
            R_k1 = R_k
            Z_k1 = Z_k
            X_k1 = X_k
            denominator = (R_k2 * Z_k2).sum(0)
            denominator[denominator == 0] = 1e-8
            beta = (R_k1 * Z_k1).sum(0) / denominator
            P_k = Z_k1 + beta.unsqueeze(0) * P_k1

        denominator = (P_k * (A@P_k)).sum(0)
        denominator[denominator == 0] = 1e-8
        alpha = (R_k1 * Z_k1).sum(0) / denominator
        X_k = X_k1 + alpha.unsqueeze(0) * P_k
        R_k = R_k1 - alpha.unsqueeze(0) * (A@P_k)
        R_k = R_k * A_supp
        end_iter = time.perf_counter()

        residual_norm = torch.norm(A@X_k - B, dim=1)

        if verbose:
            print("%03d | %8.4e" %
                    (k, torch.max(residual_norm/B_norm)))

        if (residual_norm <= stopping_matrix).all():
            optimal = True
            break


    end = time.perf_counter()

    if verbose:
        if optimal:
            print("Terminated in %d steps (optimal). Took %.3f ms." %
                    (k, (end - start) * 1000))
        else:
            print("Terminated in %d steps (reached maxiter). Took %.3f ms." %
                    (k, (end - start) * 1000))


    info = {
        "niter": k,
        "optimal": optimal
    }

    return X_k


test_x = test_inps[0]
test_y = test_outs[0][0]

H_A = torch.zeros((test_x.shape[-2], test_x.shape[-2]), device="cuda")
H_B = torch.zeros((test_y.shape[-2], test_x.shape[-2]), device="cuda")
nsamples = 0
for i in range(120):
    H_A *= nsamples / (nsamples + 1)
    H_B *= nsamples / (nsamples + 1)

    nsamples += 1

    H_A += (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T
    H_B += (math.sqrt(2 / nsamples) * test_outs[i][0].cuda().to(torch.float32)) @ (math.sqrt(2 / nsamples) * test_inps[i].cuda().to(torch.float32)).T

    

M = (new_W == 0).to(torch.float32).to(H_B.device)
print(M.device)
# M = torch.zeros_like(M).to(torch.float32).cuda()
print((M == 0).sum() / M.numel())
W_old = new_W.clone().to(H_B.device)
beta = 0.01
alpha = 0.9
gama = 0.0000
rho = 1

X_norm = torch.diag(H_A).sqrt()

H_A = H_A / X_norm
H_A = (H_A.T /X_norm).T
H_B = torch.matmul(ora_W.to(torch.float32) * X_norm, H_A)
B = (W_old * X_norm).T

start = time.time()

W_update = cg_batch(H_A, H_B, (W_old != 0).to(torch.float32), X0=B, rtol=1e-4, atol=0., maxiter=1000, verbose=False)
W_update = W_update.T / X_norm
print('time', time.time() - start)

cuda:0
tensor(0.3000, device='cuda:0')
time 40.19992709159851


In [40]:
print("Final W_update", W_update.abs().max(), W_update.abs().mean())
print((W_update == 0).sum() / W_update.numel())
print("new_W", new_W.abs().max(), new_W.abs().mean())

Final W_update tensor(12.9766, device='cuda:0', dtype=torch.float16) tensor(0.0558, device='cuda:0', dtype=torch.float16)
tensor(0.4901, device='cuda:0')
new_W tensor(0.2256, device='cuda:0', dtype=torch.float16) tensor(0.0073, device='cuda:0', dtype=torch.float16)


In [39]:
W_update = W_update.to(torch.float16)
print((W_update == 0).sum() / W_update.numel())
for i in [0, 32, 63, 122, 123, 124]:
    # W_update = W_update - M.to(torch.float16) * W_update
    print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print(i, (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    print(i, (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (new_W.to(H_B.device) @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # W_update2 = W_update - M.to(torch.float16) * W_update
    # print(i, (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().max(), (W_update2 @ test_inps[i].cuda() - test_outs[i][0].cuda()).abs().mean())
    # print("-----------")
    # print((new_W @ test_inps[i]).abs().max(), (new_W @ test_inps[i]).abs().mean())
    # print((W_update2 @ test_inps[i]).abs().max(), (W_update2 @ test_inps[i]).abs().mean())
    # print(test_outs[i][0].abs().max(), test_outs[i][0].abs().mean())
    print("-------------------------------------------------------------------------")

tensor(0.4901, device='cuda:0')
tensor(3.9316, dtype=torch.float16) tensor(0.2715, dtype=torch.float16)
0 tensor(42.9062, device='cuda:0', dtype=torch.float16) tensor(1.1484, device='cuda:0', dtype=torch.float16)
0 tensor(0.5742, device='cuda:0', dtype=torch.float16) tensor(0.0740, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(5.1016, dtype=torch.float16) tensor(0.2737, dtype=torch.float16)
32 tensor(37.8438, device='cuda:0', dtype=torch.float16) tensor(1.1699, device='cuda:0', dtype=torch.float16)
32 tensor(0.8164, device='cuda:0', dtype=torch.float16) tensor(0.0758, device='cuda:0', dtype=torch.float16)
-------------------------------------------------------------------------
tensor(3.5605, dtype=torch.float16) tensor(0.2708, dtype=torch.float16)
63 tensor(49.8750, device='cuda:0', dtype=torch.float16) tensor(1.2998, device='cuda:0', dtype=torch.float16)
63 tensor(0.6562, device='cuda:0', dtype=torch.float16) te